# Numerical Methods for Manu Body Physics, Assignment #2

Yoav Zack, ID 211677398

## Question 1: Entanglement entropy

We want to calculate the entanglement entropy of a random state under the $A\otimes B$ partition. A random state of the whole system is:
$$\ket{\psi} = \sum_{k} c_{k} \ket{k}$$
But under the bi-partition $A\otimes B$  a state is of the following form:
$$\ket{\psi} = \sum_{i,j} \Psi_{i,j} \ket{i} \ket{j}$$
Where $\ket{i}, \ket{j}$ are elements in a basis for $A$ and $B$ respectively. So, to get such a representation of the state  we will write:
$$\sum_{k} c_{k} \ket{k} = \sum_{i,j} \Psi_{i,j} \ket{i} \ket{j}$$
Assuming the original basis $\ket{k}$ is the standard spin basis and the new ones $\ket{i}, \ket{j}$ are the same but for the respective subsystems, we can write each state $\ket{k}$ as $\ket{i} \otimes \ket{j}$, where:
$$i=\sum_{n=1}^{L/2} 2^{k[2n]}\; ; \; i=\sum_{n=1}^{L/2} 2^{k[2n-1]}$$
where $k[n]$ is the $n$-th bit of the number $k$. Using this, we can write an expression for the matrix $\Psi$ as $\Psi_{i,j} = c_k$ assuming $i,j$ are defined as above.

After all of this is done, we can calculate the Von Neumann Entanglemnt entopy of the system by performing a Schmidt Decomposition (i.e. SVD) on the matrix $\Psi_{ij}$, an summing the singular values $\lambda_i$ in the following form:
$$S = -\sum_{i}\lambda_{i} \log \lambda_{i}$$

Let's do that in `julia`. First we import useful libraries:

In [ ]:
using LinearAlgebra
using TensorOperations
using Statistics
using Plots
theme(:dracula)

Then we define the bipartition of the wavefunction into two staggared chains:

In [ ]:
function index2state(stateind::Integer, N::Integer)
  return digits(stateind, base=2, pad=N)
end

In [ ]:
function staggared_bipartition(ψ::Vector{<:Number})
    @assert abs(norm(ψ) - 1) < 1e-9 "State not normalized"
    D = length(ψ)
    L = log2(D)
    @assert isinteger(L) "Number of states must be a power of two"
    L = Int(L)
    @assert isinteger(L/2) "Number of spins must be even"

    r = sqrt(D)
    @assert isinteger(r) "Number of states must be a square number"
    r = Int(r)
    
    Ψ = zeros(r, r)
    for k in range(0, length=D)
        i = sum([(typeof(k)(1) << (2*n  ) & k) >> (n  ) for n in range(0, length=Int(L/2))])
        j = sum([(typeof(k)(1) << (2*n+1) & k) >> (n+1) for n in range(0, length=Int(L/2))])
        # @show index2state(k, L), index2state(i, L), index2state(j, L)
        # @show k, i, j
        Ψ[i+1,j+1] = ψ[k+1]
    end
    return Ψ
end

And we define function which takes a wavefunction in the form of a vector and a bipartition function, performs the bipartition and calcualtes the Von-Neumann Entropy for that bypartition:

In [ ]:
function vn_entropy(ψ::Vector{<:Number}, bipartition::Function)
    Ψ = bipartition(ψ)
    return vn_entropy(Ψ)
end

function vn_entropy(Ψ::Matrix{<:Number})
    S = svdvals(Ψ)
    return sum(-S[S.!=0].^2 .* log.(S[S.!=0].^2))
end

We can test this function on all of the Bell States:

In [ ]:
@show vn_entropy([1,0,0,1]/sqrt(2), staggared_bipartition);
@show vn_entropy([1,0,0,-1]/sqrt(2), staggared_bipartition);
@show vn_entropy([0,1,1,0]/sqrt(2), staggared_bipartition);
@show vn_entropy([0,1,-1,0]/sqrt(2), staggared_bipartition);
@show log(2);

And as can be seen, all of them are equal to $\ln 2$ as expected. We can also test this on a simple case of 10,000 random states on 10 spins:

In [ ]:
k = 7
L_list = range(2, step=2, length=k)
N = 1000

mean_svn = zeros(k)
std_svn = zeros(k)

plt = histogram()
for (ind, L) in enumerate(L_list)
    svn = zeros(N)
    for iter in range(1, length=N)
        ψ = normalize(rand(2^L))
        svn[iter] = vn_entropy(ψ, staggared_bipartition)
    end
    stephist!(svn, legend=nothing, fill=true, fillalpha=0.5)
    mean_svn[ind], std_svn[ind] = mean(svn), std(svn)
end
histogram!(xlabel="Entanglement Entopy", ylabel="Count")

In [ ]:
plot(L_list, mean_svn, marker=:circle, label=nothing)

## Question 2: 1D Transverse Field Ising Model (TFIM)

In this question we will use the bond dimension $M=20$ and imaginary time step $dt=0.1$, as suggested in the assignment:

In [ ]:
maxM = 20
dt = 0.1

And also define the initial state as an Neel MPS State, using the following function from class:

In [ ]:
function NeelMPS(L::Int64)
    # Initialize an array to store matrices representing the Neel state MPS.
    ψ = Array{Float64}[]

    # Iterate through each site in the 1D quantum system.
    for i in 1:L
        # Check if the site index is odd (spin-up) or even (spin-down).
        if isodd(i)
            # For spin-up, add a matrix with coefficients [1.0, 0.0] to the array.
            push!(ψ, reshape([1.0, 0.0], (1, 2, 1)))
        else
            # For spin-down, add a matrix with coefficients [0.0, 1.0] to the array.
            push!(ψ, reshape([0.0, 1.0], (1, 2, 1)))
        end
    end

    # Return the array representing the Neel state MPS.
    return ψ
end

mutable struct MPSrep
    oc::Int64                        # orthogonality center
    maxM::Int64                      # maximal bond dimension
    state::Vector{Array{Float64}}  
    
    function MPSrep(psi::Vector{Float64}, maxM=-1)
        # initial MPS given in left-canonical form, oc at rightmost site
        state, VN, Ms, Trunc = MPSrep_LC(psi, maxM)
        oc = length(state)
        new(oc, maxM, state)
    end
    
    function MPSrep(psi::Vector{Array{Float64}})
        # initial MPS given in right-canonical form, oc at leftmost site
        oc = 1
        maxM = -1
        state = deepcopy(psi)
        new(oc, maxM, state)
    end
    
end

In order to generate the ground state of the system we wll use the iTEBD method from class, by the following functions:

In [ ]:
function SvdBondTensor!(psi::Vector{Array{Float64}}, b::Int64, psi_bond::Array{Float64}, ortho_left=true, maxM=-1, renorm=false)
    """
    SvdBondTensor!(psi::Vector{Array{Float64}}, b::Int64, psi_bond::Array{Float64}, ortho_left=true, maxM=-1, renorm=false)

    Update the Matrix Product State (MPS) by performing a Singular Value Decomposition (SVD) on the bond tensor.

    # Arguments
    - `psi`: Array of matrices representing the MPS.
    - `b`: Index of the bond between adjacent sites (1 to length(psi)-1).
    - `psi_bond`: Bond tensor connecting sites `b` and `b+1`.
    - `ortho_left`: If true, orthogonally update the matrix at site `b` (left-canonical). If false, orthogonally update the matrix at site `b+1` (right-canonical).
    - `maxM`: The maximum number of singular values to keep in each step. If -1, no truncation is performed.
    - `renorm`: If true, renormalize the singular values after the SVD.

    # Returns
    - `trunc`: Truncation error if truncation occurs, otherwise 0.

    """
    @assert length(size(psi_bond)) == 4 "psi_bond is expected to be a rank-4 tensor"

    trunc = 0.

    M0 = size(psi_bond)[1]
    M2 = size(psi_bond)[4]

    # Reshape the bond tensor into a matrix for SVD
    psi_bond_matrix = reshape(psi_bond, (M0*2, M2*2))

    # Perform SVD on the reshaped bond tensor matrix
    U, S, V = svd(psi_bond_matrix)

    # Optionally renormalize the singular values
    if renorm
        S /= norm(S)
    end

    # Find the last non-zero singular value
    M1 = findlast(S .> 1e-16)

    # Retain only relevant parts of U, S, and V
    U, S, V = U[:, 1:M1], S[1:M1], V[:, 1:M1]

    # If 0 < maxM < M1 - truncation, perform truncation
    if maxM > 0 && M1 > maxM
        trunc = sum(S[maxM:end].^2)
        U, S, V = U[:, 1:maxM], S[1:maxM], V[:, 1:maxM]
        S /= sqrt(1. - trunc)
        M1 = maxM
    end

    # Update the MPS matrices based on the SVD results
    if ortho_left
        psi[b] = reshape(U, (M0, 2, M1))
        psi[b+1] = reshape(Diagonal(S) * V', (M1, 2, M2))
    else
        psi[b] = reshape(U * Diagonal(S), (M0, 2, M1))
        psi[b+1] = reshape(V', (M1, 2, M2))
    end

    return trunc
end


function BondTensor(psi::Vector{Array{Float64}}, b::Int64)
    """
    BondTensor(psi::Vector{Array{Float64}}, b::Int64)

    Construct the bond tensor between two adjacent sites in a Matrix Product State (MPS).

    # Arguments
    - `psi`: Array of matrices representing the MPS.
    - `b`: Index of the bond between adjacent sites (1 to length(psi)-1).

    # Returns
    - `psi_bond`: Bond tensor connecting sites `b` and `b+1`.

    """
    @assert b >= 1 && b <= (length(psi)-1) "Bond out of range"

    # Dimensions of matrices at site b and site b+1
    M0 = size(psi[b])[1]
    M2 = size(psi[b+1])[3]

    # Initialize the bond tensor
    psi_bond = zeros(M0, 2, 2, M2)

    # Contract matrices at site b and site b+1 to form the bond tensor
    @tensor psi_bond[l0, s1, s2, l2] = psi[b][l0, s1, l1] * psi[b+1][l1, s2, l2]

    return psi_bond
end


function BondH(h1::Float64, h2::Float64)
    """
    BondH(h1::Float64, h2::Float64)

    Generate the bond Hamiltonian matrix for a two-site bond in the Transverse Field Ising Model (TFIM).

    # Arguments
    - `h1`: Coefficient for the Sx⨂Id term.
    - `h2`: Coefficient for the Id⨂Sx term.

    # Returns
    The Hamiltonian matrix for the two-site bond in the TFIM.

    """
    # Pauli matrices
    Sx = [0.0 1.0; 1.0 0.0]
    Sz = [1.0 0.0; 0.0 -1.0]
    Id = [1.0 0.0; 0.0 1.0]

    # Construct the Hamiltonian matrix for the two-site bond in the TFIM
    Hb = -kron(Sz, Sz) + h1 * kron(Sx, Id) + h2 * kron(Id, Sx) # -------------------------------------------------------------------------------------
    
    return Hb
end

function ApplyGate!(psi::Vector{Array{Float64}}, op::Array{Float64}, b::Int64, ortho_left=true, maxM=-1, renorm=false)
    """
    ApplyGate!(psi::Vector{Array{Float64}}, op::Array{Float64}, b::Int64, ortho_left=true, maxM=-1, renorm=false)

    Apply a gate to a specific bond in the Matrix Product State (MPS) representation.

    # Arguments
    - `psi`: Array of matrices representing the MPS.
    - `op`: Array representing the gate to be applied.
    - `b`: Index of the bond where the gate is applied.
    - `ortho_left`: If true, update the left canonical form after applying the gate. Default is true.
    - `maxM`: Maximum bond dimension. If -1, no truncation is performed. Default is -1.
    - `renorm`: If true, renormalize singular values after applying the gate. Default is false.

    # Returns
    The truncation error if truncation is performed.

    """
    trunc = 0.

    # Calculate the bond tensor for the current bond
    psi_bond = BondTensor(psi, b)

    # Initialize a new bond tensor after applying the gate
    new_psi_bond = zeros(size(psi_bond))
    
    # Contract the gate with the current bond tensor to obtain the new bond tensor
    @tensor new_psi_bond[l0, s1p, s2p, l2] = op[s2p, s1p, s2, s1] * psi_bond[l0, s1, s2, l2]

    # Update the MPS by applying the new bond tensor
    trunc = SvdBondTensor!(psi, b, new_psi_bond, ortho_left, maxM, renorm)

    return trunc
end

function GateTensor(dt::Float64, h1::Float64, h2::Float64)
    """
    GateTensor(dt::Float64, h1::Float64, h2::Float64)

    Generate the gate tensor for a time evolution step in the Transverse Field Ising Model (TFIM).

    # Arguments
    - `dt`: Time step for the evolution.
    - `h1`: Coefficient for the Sx⨂Id term.
    - `h2`: Coefficient for the Id⨂Sx term.

    # Returns
    The gate tensor for the time evolution step in the TFIM.

    """
    # Generate the Hamiltonian matrix for the two-site bond in the TFIM
    Hb = BondH(h1, h2)

    # Construct the gate tensor for the time evolution step using the Hamiltonian
    G = exp(-dt/2 * Hb)

    # Reshape the gate tensor to a 4-index tensor
    return reshape(G, (2, 2, 2, 2))
end

function iTEBD_sweep!(psi::Vector{Array{Float64}}, dt::Float64, h::Float64, maxM::Int64)
    """
    iTEBD_sweep!(psi::Vector{Array{Float64}}, dt::Float64, h::Float64, maxM::Int64)

    Perform a single sweep of the imaginary time evolution using Time-Evolving Block Decimation (iTEBD).

    # Arguments
    - `psi`: Array of matrices representing the Matrix Product State (MPS).
    - `dt`: Time step for the evolution.
    - `h`: Transverse field strength.
    - `maxM`: Maximum bond dimension.

    # Returns
    The maximum truncation error during the sweep.

    """
    L = length(psi)
    
    maxTrunc = 0.

    # Sweep right
    for i in range(1, stop=L-1)
        h1 = i == 1 ? h : h / 2
        h2 = i == L-1 ? h : h / 2
        gateOp = GateTensor(dt, h1, h2)
        
        # Apply the gate, update orthogonality center to the right, and renormalize
        trunc = ApplyGate!(psi, gateOp, i, true, maxM, true)
        
        # Update the maximum truncation error
        if trunc > maxTrunc
            maxTrunc = trunc
        end
    end
    
    # Sweep left
    for i in range(L-1, stop=1, step=-1)
        h1 = i == 1 ? h : h / 2
        h2 = i == L-1 ? h : h / 2
        gateOp = GateTensor(dt, h1, h2)
        
        # Apply the gate, update orthogonality center to the left, and renormalize
        trunc = ApplyGate!(psi, gateOp, i, false, maxM, true)
        
        # Update the maximum truncation error
        if trunc > maxTrunc
            maxTrunc = trunc
        end
    end
    
    return maxTrunc
end

function OrthogonalizePsi!(psi_mps::MPSrep, i0::Int64)
    """
    OrthogonalizePsi!(psi_mps::MPSrep, i0::Int64)

    Bring the orthogonality center to site i0 in a Matrix Product State (MPS), updating the MPS representation.

    # Arguments
    - `psi_mps`: An object representing the MPS.
    - `i0`: The target site to which the orthogonality center (oc) should be moved.

    """
    L = length(psi_mps.state)
    oc = psi_mps.oc

    if i0 > oc
        # Move the orthogonality center to the right: Add more left canonical forms
        for i in oc:i0-1
            psi_bond = BondTensor(psi_mps.state, i)
            # ortho_left = true => Update the matrix at site i (left canonical form)
            # maxM = -1 => No truncation
            # renormalize = false => Do not renormalize the singular values
            SvdBondTensor!(psi_mps.state, i, psi_bond, true)
        end
    elseif i0 < oc
        # Move the orthogonality center to the left
        for i in oc-1:-1:i0
            psi_bond = BondTensor(psi_mps.state, i)
            # ortho_left = false => Update the matrix at site i+1 (right canonical form)
            # maxM = -1 => No truncation
            # renormalize = false => Do not renormalize the singular values
            SvdBondTensor!(psi_mps.state, i, psi_bond, false)
        end
    end

    # Update the orthogonality center
    psi_mps.oc = i0
end

function ExpectationValue(psi_mps::MPSrep, Op)
    """
    ExpectationValue(psi_mps::MPSrep, Op)

    Calculate the expectation values of an operator (Op) for each site in a Matrix Product State (MPS).

    # Arguments
    - `psi_mps`: An object representing the MPS.
    - `Op`: The operator for which to calculate the expectation values.

    # Returns
    An array of expectation values for each site.

    """
    L = length(psi_mps.state)
    ExpVals = zeros(L)

    for i in 1:L
        # Orthogonalize the MPS up to site i
        OrthogonalizePsi!(psi_mps, i)

        # Extract the matrix Ai from the MPS representation at site i
        Ai = psi_mps.state[i]
        M0 = size(Ai)[1]
        M1 = size(Ai)[3]

        # Initialize the tensor representing Ai * Op
        Ai_Sop = zeros(M0, 2, M1)

        # Contract Ai with Op to obtain Ai_Sop tensor
        @tensor Ai_Sop[l0, sp, l1] = Ai[l0, s, l1] * Op[s, sp]

        # Calculate the expectation value
        @tensor Val = Ai_Sop[l0, sp, l1] * Ai[l0, sp, l1]
        ExpVals[i] = Val
    end

    return ExpVals
end

### Section 2.1: Ferromagnetic phase

In this case the magnetic field to interaction ratio is $h/J=0.5$:

In [ ]:
J = 1.0;
h = 0.5;

We will start by getting the ground state using iTEBD sweeps:

In [ ]:
L = 100
beta = 10

Nt = Int(beta/dt)
ψ = NeelMPS(L)
ψ_ground = MPSrep(ψ)
for ti in range(1, length=Nt)
    trunc = iTEBD_sweep!(ψ_ground.state, dt, h, maxM)
end

To get the expectation value of $S_z$ along the chain we will use the `ExpectationValue()` function from class and the $S_z$ operator:

In [ ]:
Sz = [1.0 0.0; 0.0 -1.0]
SzVals = ExpectationValue(ψ_ground, Sz)
plot(SzVals)

And we can see that the correlation is about constant and about zero, which is lower then I expected, but generally not surprising, since the magnetic field $h$ makes the system not completely correlated. And as evidence:

In [ ]:
h = 0.0
Nt = Int(beta/dt)
ψ = NeelMPS(L)
ψ_ground = MPSrep(ψ)
for ti in range(1, length=Nt)
    trunc = iTEBD_sweep!(ψ_ground.state, dt, h, maxM)
end
SzVals = ExpectationValue(ψ_ground, Sz)
plot(SzVals)

When $h=0$ we get a complete ferromagnet as expected